# 🦜🔗 LangChain核心源代码解读（下）

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# （二）解读源码，深度定制自己的智能体

## 下半场的闲聊

- 🌹 [LangChain 中的 Agent](https://python.langchain.com/docs/modules/agents/agent_types/)
- 🌹 [Chain 遗产](https://python.langchain.com/docs/modules/chains)

### 🌹 智能体相关类整理

**（1）智能体类型**
- AgentType
    - ZERO_SHOT_REACT_DESCRIPTION（`ReAct`的一般实现）
    - REACT_DOCSTORE（ReAct，支持RAG）
    - SELF_ASK_WITH_SEARCH（使用`search 工具`不断反思获得答案）
    - CONVERSATIONAL_REACT_DESCRIPTION（`ReAct`，支持对话）
    - CHAT_ZERO_SHOT_REACT_DESCRIPTION（同上）
    - CHAT_CONVERSATIONAL_REACT_DESCRIPTION（同上）
    - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION（`ReAct`，为对话模型优化，支持多输入）
    - OPENAI_FUNCTIONS（支持`OpenAI Function Calling`）
    - OPENAI_MULTI_FUNCTIONS（支持`OpenAI Function Calling`，支持多函数调度）

**（2）智能体执行器子组件：**

- `AgentAction`
    - `AgentActionMessageLog`
- `AgentStep`
- `AgentFinish`

**（3）单动智能体：**
- `BaseSingleActionAgent`
    - RunnableAgent
    - LLMSingleActionAgent（__deprecated__：`create_***_agent`）
    - XMLAgent（__deprecated__：create_xml_agent）
    - Agent
        - ChatAgent（__deprecated__：`create_react_agent`）
        - ConversationalAgent（__deprecated__：`create_react_agent`）
        - ConversationalChatAgent（__deprecated__：`create_json_chat_agent`）
        - StructuredChatAgent（__deprecated__：`create_structured_chat_agent`）
        - ZeroShotAgent（__deprecated__：`create_react_agent`）
        - ReActDocstoreAgent（__deprecated__）
            - ReActTextWorldAgent（__deprecated__）
        - SelfAskWithSearchAgent（__deprecated__：`create_self_ask_with_search`）
    - OpenAIFunctionsAgent（__deprecated__：`create_openai_functions_agent`）

**（4）多动智能体：**
- BaseMultiActionAgent
    - RunnableMultiActionAgent
    - OpenAIMultiFunctionsAgent（__deprecated__：`create_openai_tools_agent`）

**（5）Assistant：**
- Runnable
    - RunnableSerializable
        - Chain
            - `AgentExecutor`
                - MRKLChain（__deprecated__）
                - ReActChain（__deprecated__）
                - SelfAskWithSearchChain（__deprecated__）
        - `OpenAIAssistantRunnable`

**（6）提示语模板：**
- Runnable
    - RunnableSerializable
        - BasePromptTemplate [Dict, PromptValue]
            - BaseChatPromptTemplate
                - ChatPromptTemplate
                    - `AgentScratchPadChatPromptTemplate`

**（7）输出解析：**
- Runnable
    - RunnableSerializable
        - BaseOutputParser
            - `AgentOutputParser`
            - `MultiActionAgentOutputParser`

### 🌹 从源码中搜寻 openai 的痕迹

- [partners/openai/langchain_openai/__init__.py](https://github.com/langchain-ai/langchain/blob/master/libs/partners/openai/langchain_openai/__init__.py)
- [core/langchain_core/utils/function_calling.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/utils/function_calling.py)
- [core/langchain_core/output_parsers/openai_tools.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_tools.py)
- [core/langchain_core/output_parsers/openai_functions.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_functions.py)

## 4、 openai风格智能体

### 🌹 Function Calling 是怎么回事

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    convert_to_openai_function 和 convert_to_openai_tool 的区别是什么？
</div>

**🌞 参考：**
- [查看 convert_to_openai_tool 的实现源码](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/utils/function_calling.py#L323-L341)


**回顾上面的例子：**

In [2]:
from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool, convert_to_openai_function
import re

@tool
def ask_neighber(query: str) -> str:
    """我在玩找人的游戏，我知道你找的人住在哪个房间"""
    if(re.search("马冬梅", query)):
        return "楼上322"
    else:
        return "我不清楚"

In [3]:
convert_to_openai_function(ask_neighber)

{'name': 'ask_neighber',
 'description': 'ask_neighber(query: str) -> str - 我在玩找人的游戏，我知道你找的人住在哪个房间',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [4]:
convert_to_openai_tool(ask_neighber)

{'type': 'function',
 'function': {'name': 'ask_neighber',
  'description': 'ask_neighber(query: str) -> str - 我在玩找人的游戏，我知道你找的人住在哪个房间',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string'}},
   'required': ['query']}}}

### 🌹 阅读 create_openai_tool 源码，解读运行过程

[langchain/agents/openai_tools/base.py](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/openai_tools/base.py#L15-L97)

```python
def create_openai_tools_agent(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate
) -> Runnable:
    """Create an agent that uses OpenAI tools."""

    missing_vars = {"agent_scratchpad"}.difference(prompt.input_variables)
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    llm_with_tools = llm.bind(tools=[convert_to_openai_tool(tool) for tool in tools])

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            )
        )
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
    )
    return agent
```

### ✍️ 最原始的版本

### ✍️ 代码复用的版本

## 5、 react风格智能体

### 🌹 阅读 create_react_agent 源码，解读运行过程

- 比原生 Function-Calling 多了必要的解析过程
- 基于提示语可以构建类似原生支持 Function-Calling 的智能体运行逻辑吗？

### ✍️ 一步一步测试 create_react_agent 过程

### ✍️ 自定义一个 ReAct 智能体

- 一个简单的智能体需求：与AI玩一个捉迷藏游戏
- react智能体的prompt如何工作
- 中间步骤的一步步产生过程
- action如何解析

## 6、在LCEL框架下重新《手撕AutoGPT》

### ✍️ 自定义《手撕AutoGPT》智能体的提示语
### ✍️ 自定义一个智能体 OutputParser

- 工具：定义一个简单工具
- 智能体：
    - Tools-Calling 智能体
    - ReAct 智能体
- 执行器：AgentExecutor

难点：
- 官方例子和内置智能体无法支持pydantic参数解析（智谱AI等推理能力较弱的模型可以使用）

### 🌹 阅读 AgentExcutor 源码：流支持

### 🌹 AgentExcutor中的记忆体

## 7、Langgraph中的惊喜

**一起来打开眼界**

列举示例中对各种新的智能体论文实践...

# 结束了

## ❤️ 总结

- 我们一起阅读了langchain的源代码结构和部份细节
    - BaseLanguageModel / BaseChatModel / 
    - Runnable
    - LambdaRunnable
    - Chain
    - AgentExcutor
    - langgraph.prebuild
- 我们学习了如何自己动手集成大模型到 langchain 中
- 我们拆解了langchain的基石组件：Runnable
- 我们拆解了langchain的核心逻辑能力：LCEL
- 我们拆解了langchain的最新逻辑能力：langgraph
- 我们动手做了一些代码实践